# RAG Task - PwC Junior AI Engineer Application Task

## Intro

This notebook demonstrates my solution at creating an agentic chatbot with autonomous decision making, retrieval-augmented generation (RAG) and integration of external knowledge via FAISS.
Although the notebook contains `.py` code snippets, the source code is logically separated into 2 Python files - one responsible for executing the LangGraph-based chatbot (langGraphProt.py), and the other responsible for vectorizing datasets (dataLoader.py). This is so that if new datasets are to be added (or the current one is to be expanded), it can be easily done without messing with the main program. 

## Dependencies

Install all required libraries:

`pip install faiss-cpu numpy torch sentence-transformers PyPDF2 requests`

## Design Decisions
- **LLM**: For choosing a Large Language Model, I had to balance reasoning ability, context window size, parameter size, latency, memory requirement and quality of generation, all while ensuring the model is free to use. Since PwC users’ queries may require summarization, handling long documents, or multi-step reasoning, I needed a model with strong long-text and reasoning performance that remains accessible for prototyping. Initially I chose DeepSeek R1, but after some mixed-quality / inconsistent answers for English questions, I decided to switch model. I chose to work with keys available on [OpenRouter](https://openrouter.ai/). My final choice was my alternative, the [model](https://openrouter.ai/meta-llama/llama-3.3-70b-instruct:free) Meta Llama 70B, because multiple suitable models that I have tried working with were either rate-limited, or were throwing error, saying that my query is flagged unexpectedly. However, this model perfectly meets the requirements I described above, and was suitable for me for prototyping.
- **Embedding model**: I have selected an embedding model that was present on the HuggingFace website's [leaderboard](https://huggingface.co/spaces/mteb/leaderboard). I wanted a model with good semantic capture at a reasonable cost (latency and storage). So I choose a model with 0.6B parameter size that is feasible to work with on my laptop, but is still good for prototyping. My choice is the [Qwen3-Embeddings-0.6B](https://huggingface.co/Qwen/Qwen3-Embedding-0.6B) with 595M parameters, 1024-dimension vectors, solid retrieval time, but still a lightweight model. 
- **FAISS**: I chose [FAISS](https://faiss.ai/index.html) because it is an efficient, lightweight and is a widely used library for similarity search on dense vectors. It allows me to quickly retrieve relevant chunks of texts using vector embeddings. Since scalability is crucial, FAISS is a strong choice because it can handle millions of vectors really quickly and can be easily integrated with Python.
- **Pipeline**: I designed a pipeline as a state graph with clearly defined nodes (input -> retriever -> controller -> LLM -> output) to demonstrate agentic behaviour and modularity. While this structure is simple, it efficiently showcases autonomous decisionmaking and keeps the prototype scalable and maintainable. 

## Reflections
- **Performance**: I could test the performance of the chatbot in two steps: first is by measuring the speed of embedding the dataset, and the other is by measuring time it takes for individual nodes in the pipeline to execute code. As for the former, vectorization takes about 20 seconds on average, with the nodes in the main program having to execute for differing amounts of time. In most cases during development, the LLM node took the longest to execute, due to having to make an API call and receive a formulated response. Other nodes took about 1 second to execute on average.   

- **Bottlenecks**:
1. Embedding generation: chunk embeddings are precomputed, but but query embeddings are computed at runtime. We can use a lightweight embedding model for queries to decrease latency, and/or cache repeated queries.
2. FAISS search: usually fast, but with large datasets the search time increases and memory usage can spike.
3. LLM call: due to network latency + model inference time. Using a large remote model is the slowest part.
4. Controller retries: each retry doubles computation, increasing latency
5. State printing: JSON dumps can slow down debug output 
- **Future improvements**:
1. Embedding model: Currently, it is lightweight, free, and runs locally. In the future, multilingual embedding models can be used that offer better semantic accuracy, broader language coverage and larger context window. They would run on decentralized servers with GPUs (PwC cloud for example) or specialized vector database services (Pinecone for example).
2. LLM: Currently, it is lightweight and free, and runs externally. In the future, multilingual LLMs can be used with stronger reasoning abilities that would run on PwC's secure cloud environment (for compliance) or via private API gateways (so data never leaves PwC infrastructure). 
3. Retrieval: Currently, the dataset is small and local. In the future, we can increease the RAG dataset (more documents / chunks, and web access), and can increase the retrieval accuracy via better embedding models, more sophisticated chunking, multi-hop retrieval and by adding metadata. 
4. Pipeline: In the future, we can integrate sources into answers, can include logging or state visualization, and can improve the controller by a smarter coverage thresholding and conditional retries. 

## Source Codes

As said, there are two python files - `dataLoader.py` and `langGraphProt.py`, which can also be found in `src/`. 

`dataLoader.py`:

In [ ]:
import PyPDF2
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
import faiss
import json


##ADJUSTABLE PARAMETERS##
filename = "1706.03762v7.pdf"
#In this case, it can be replaced with any PDF
embeddingModel = SentenceTransformer("models/qwen/Qwen3-Embedding-0.6B")
#Load the embedding model
chunkS = 500
#Preferred chunk size
overl = 50
#Preferred overlap



#Function that turns PDF into text using the PyPDF2 library
def turnToText(path):
    read = PyPDF2.PdfReader(path)
    text = ""
    for page in read.pages:
        #Extract page by page
        text += page.extract_text() + "\n"
    return text


#Function that splits the extracted text into chunks
def splitter(text):
    chunkSize = chunkS
    overlap = overl
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunkSize - overlap):
        chunk = " ".join(words[i:i + chunkSize])
        chunks.append(chunk)
    return chunks


chunks = [] 
#To ensure it will not be ran (causing high latency) during import
if __name__ == "__main__": 
    pdfText = turnToText(filename)
    chunks = splitter(pdfText)
    #Create chunks of text of a given file

    chunk_vectors = []
    for chunk in chunks:
        #Vectorize chunks using the embedding model
        vector = embeddingModel.encode(chunk, convert_to_numpy=True)
        chunk_vectors.append(vector)
    chunk_vectors = np.array(chunk_vectors).astype("float32") 
    #FAISS needs float32

    with open("src/faiss_index/chunks.json", "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2) 
        #Save chunks as texts too

    dimensions = chunk_vectors.shape[1]
    index = faiss.IndexFlatL2(dimensions)
    #Grab vector size (dimensions) and index with it
    index.add(chunk_vectors)

    faiss.write_index(index, "src/faiss_index/chunk_index.faiss")
    print("FAISS saved")
    #Write FAISS to disk

`langGraphProt.py`:

In [ ]:
print("Import initialized.")
import time
startImp = time.perf_counter()
import os
import requests
print("Loading FAISS and embedding model, please wait...")
import faiss
from dataLoader import embeddingModel, chunks
import numpy as np
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import json
from langchain_core.messages import HumanMessage, AIMessage
from sklearn.metrics.pairwise import cosine_similarity
import re
endImp = time.perf_counter()
print(f"Import successful (took {endImp-startImp:.1f} seconds). Initializing graph...")
#At this point, all imports are done (timed for performance)



##ADJUSTABLE PARAMETERS##
os.environ["API_KEY"] = "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"
#Replace with your API key
SITE_URL = "https://openrouter.ai/api/v1/chat/completions"
#Replace with a preferred site URL
langModel = "meta-llama/llama-3.3-70b-instruct:free"
#Replace with a preferred model
kValue = 5
#Replace with a preferred amount of chunks (context) to fetch
minCoverage = 0.7
#Replace with a preferred minimum of keyword coverage in the retrieved chunks
maxDistance = 1.2
#Replace with a preferred max distance of similarity



faissIndexPath = "src/faiss_index/chunk_index.faiss"
#Contains FAISS index, used for quick and efficient similarity search - created upon running dataLoader.py
index = faiss.read_index(faissIndexPath)
with open("src/faiss_index/chunks.json", "r", encoding="utf-8") as f:
    chunk_texts = json.load(f)
    #Read the chunks as text too to pass on to the LLM alongside with the query
print(f"Chunks successfully loaded.")


class State(TypedDict):
    messages: Annotated[list, add_messages] 
    retrieval: dict
graph_builder = StateGraph(State)


#A simple function to retrieve keywords from a query
def getKeywords(text):
    commonWords = {"a", "an", "the", "is", "and", "or", "for", "to", "with", "of", "on", "in"}
    #Words to ignore
    allWords = re.findall(r"\w+", text.lower())
    keywords = [w for w in allWords if w not in commonWords]
    #Retrieve keywords
    return keywords
#A simple function that returns the percentage of keywords covered in the retrieved chunks
def keywordCoverage(query, chunks):
    keywords = set(getKeywords(query))
    #Retrieve keywords using the above function
    chunk_text = " ".join(chunks).lower()
    matched = [kw for kw in keywords if kw in chunk_text]
    #Get words that match and return the ratio (percentage of covered keywords)
    return len(matched) / (len(keywords) or 1)


#The first node in the pipeline
def input_node(state: State):
    query = input("(CTRL+C to quit) Enter your question: ").strip()
    state["messages"].append(HumanMessage(content=query))
    #Takes the input from the user, appends it into the state corresponding to messages,
    #and returns it to the next node: the Retriever
    return state
graph_builder.add_node("input", input_node)


#The second node in the pipeline, responsible for fetching chunks relevant to the user query
#From here on, all time-consuming nodes are timed to measure performance
def retriever_node(state: State):
    start = time.perf_counter()
    query = state["messages"][-1].content
    #Retrieve user query
    query_vector = embeddingModel.encode(query, convert_to_numpy=True).astype("float32")
    query_vector = np.expand_dims(query_vector, axis=0)
    #Vectorize the user query using the embedding model
    distances, indices = index.search(query_vector, kValue)
    chunks = [chunk_texts[i] for i in indices[0] if i<len(chunk_texts)]
    #Retrieve top k similar chunks (similarity determined via distance)
    state["retrieval"] = {
        "chunks": chunks, "distances": distances[0].tolist()
    }
    #Update the state with the chunks
    end = time.perf_counter()
    print(f"RETRIEVER node took {end-start:.1f} seconds")
    return state
graph_builder.add_node("retriever", retriever_node)


#Auxiliary function to reformulate a given query using the selected LLM
def reformulateQuery(query):
    start = time.perf_counter()
    #Construct header of the message
    headers = {
        "Authorization": f"Bearer {os.environ['API_KEY']}",
        "Content-Type": "application/json"
    }
    #Construct payload (chosen model and rewriting task with the given query)
    payload = {
        "model": langModel,
        "messages": [
            {"role": "system", "content": "You are an assistant that rewrites user queries into cleaner search queries"},
            {"role": "user", "content": f"Rewrite this question so it works better for searching documents: {query}"}
        ]
    }
    #Post the question and obtain response message 
    resp = requests.post(SITE_URL, json=payload, headers=headers)
    result = resp.json()

    try:
        toResult = result["choices"][0]["message"]["content"].strip()
        print(f"Rewrote question: {toResult}")
        #Extract response (rewrote question)
        end = time.perf_counter()
        print(f"REFORMULATOR took {end-start:.1f} seconds")
        return toResult
    except:
        print("Failed to rewrite query")
        #If error, keep original message
        end = time.perf_counter()
        print(f"REFORMULATOR took {end-start:.1f} seconds")
        return query


#The third node in the pipeline, responsible for deciding if more relevant chunks are required (and fetching them)
def controller_node(state: State):
    #Simple function to vectorize the query
    def encodeQuery(q):
        query_v = embeddingModel.encode(q, convert_to_numpy=True).astype("float32")
        return np.expand_dims(query_v, axis=0)
    start = time.perf_counter()
    query = state["messages"][-1].content
    chunks = state.get("retrieval", {}).get("chunks", [])
    #Extract query and relevant chunks
    coverage = keywordCoverage(query, chunks)
    print(f"Keyword coverage: {coverage:.2f}")
    distances = state.get("retrieval", {}).get("distances", [])
    #Calculate keyword coverage and distances

    #Best distance is used to ensure retrieval quality
    if distances:
        best_distance = min(distances)
    else:
        best_distance = 999
    print(f"Best distance is: {best_distance}")



    #If best distance is high (best chunks' relevance are low), quality is bad -> LLM fallback
    if best_distance > maxDistance:
        #Failed retrieval
        print("Low similarity, now good matches found.")
        state["retrieval"] = {"chunks": [], "distances": []}
        state["retrieval_status"] = "failed"
        state["messages"].append(AIMessage(content="No relevant context could be found."))
        #Update state accordingly, which the LLM node will see
        end = time.perf_counter()
        print(f"CONTROLLER node took {end-start:.1f} seconds")
        return state

    #If keyword coverage is low in the retrieved chunks, more chunks are needed for better context
    elif coverage < minCoverage:
        #Retrieved but low coverage -> fetching more
        print("\nCoverage is low, retrying context retrieval with double the chunks...")
        query_vector = encodeQuery(query)
        distances, indices = index.search(query_vector, kValue*2)
        #Double the number of chunks retrieved - this can be adjusted freely
        newChunks = [chunk_texts[i] for i in indices[0] if i<len(chunk_texts)]
        state["retrieval"] = {
            "chunks": newChunks, "distances": distances[0].tolist()
        }
        #Retrieve the chunks and update state

        newCoverage = keywordCoverage(query, newChunks)
        if newCoverage < minCoverage:
            #Still low coverage, reformulating question and fetching again
            print("\nNot enough coverage! Asking LLM to reformulate query.")
            refined = reformulateQuery(query)
            print(f"Refined query: {refined}")
            #Reformulate the query using the above function to potentially increase coverage

            reformulatedCoverage = keywordCoverage(refined, newChunks)
            if reformulatedCoverage < minCoverage:
                #Low coverage even with reformulated question, failed retrieval
                state["retrieval"] = {"chunks": [], "distances": []}
                state["retrieval_status"] = "failed"
                state["messages"].append(AIMessage(content="No relevant context could be found."))
                #Update state accordingly (fail)
                end = time.perf_counter()
                print(f"CONTROLLER node took {end-start:.1f} seconds")
                return state
            
            #Reformulated question has good coverage, proceed with that
            query_vector = encodeQuery(refined)
            distances, indices = index.search(query_vector, kValue*2)
            reformChunks = [chunk_texts[i] for i in indices[0] if i<len(chunk_texts)]
            #Obtain relevant chunks
            state["retrieval"] = {
                "chunks": reformChunks, "distances": distances[0].tolist()
            }
            state["messages"].append(AIMessage(content=f"Reformulated query used: {refined}"))
            state["retrieval_status"] = "success"
            #Update state accordingly (success)
    
    else:
        #Enough coverage (success)
        state["retrieval_status"] = "success"

    end = time.perf_counter()
    print(f"CONTROLLER node took {end-start:.1f} seconds")
    return state
graph_builder.add_node("controller", controller_node)


#The fourt node in the pipeline, responsible for combining fetched context with the query (RAG) and pass it on to the LLM
def llm_node(state: State):
    start = time.perf_counter()
    query = state["messages"][-1].content
    chunks = state.get("retrieval", {}).get("chunks", [])
    #Retrieve query and chunks from state 
    if not chunks:
        answer = "Could not find relevant information in the document(s). Please try rephrasing the query."
        state["messages"].append(AIMessage(content=answer))
        #No relevant chunks found
        end = time.perf_counter()
        print(f"LLM node took {end-start:.1f} seconds")
        return state
    
    #Else, join the chunks, and formulate a question to the LLM using the user question and the chunks
    context = "\n".join(chunks)
    question = f"Answer this question using the context.\n\nContext:\n{context}\n\nQuestion: {query}"
    #Construct headers and payload just like before, but with the question + chunks
    headers = {
        "Authorization": f"Bearer {os.environ['API_KEY']}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": langModel,
        "messages": [{"role": "user", "content": question}]
    }
    #Obtain response
    resp = requests.post(SITE_URL, json=payload, headers=headers)
    result = resp.json()

    #Try to get answer
    try:
        answer = result["choices"][0]["message"]["content"]
    except:
        answer = f"Error: {result}"
    #Add response message to state
    state["messages"].append(AIMessage(content=answer))
    end = time.perf_counter()
    print(f"LLM node took {end-start:.1f} seconds")
    return state
graph_builder.add_node("llm", llm_node)


#The "last" node in the pipeline (loops back to input), responsible for outputting the answer given by the LLM
#Or if there are any error messages
def output_node(state: State):
    print("\nAnswer: ")
    print(state["messages"][-1].content)
    #Simply print out the message
    return state
graph_builder.add_node("output", output_node)


#Add edges showing the direction of the pipeline (state flow)
graph_builder.add_edge(START, "input")
graph_builder.add_edge("input", "retriever")
graph_builder.add_edge("retriever", "controller")
graph_builder.add_edge("controller", "llm")
graph_builder.add_edge("llm", "output")
graph_builder.add_edge("output", "input")
#From output, loop back to input to allow more chat with the user
graph = graph_builder.compile()
#Compile the graph and initialise the first state
initial_state = State(messages=[])


if __name__ == "__main__":
    try:
        state = initial_state
        while True:
            state = graph.invoke(state)
            #Run the graph
    except KeyboardInterrupt:
        #Handle user wanting to quit, gracefully
        print("\nUser pressed CTRL+C. Exiting gracefully...")
        exit(0)